<a href="https://colab.research.google.com/github/ShawnLiu119/Coding_test_modeling_template/blob/main/Coding_Test_Template_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Coding Test Template - Classification

This code will use Heart Attack Analysis & Prediction Dataset (https://www.kaggle.com/datasets/rashikrahmanpritom/heart-attack-analysis-prediction-dataset) from Kaggle to walk through end-to-end model development, from data wrangling to feature importance output and prediction

In [2]:
import os
import numpy as np
import pandas as pd
import warnings
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
warnings.filterwarnings("ignore")
pd.set_option("display.max_rows",None)
from sklearn import preprocessing
import matplotlib
matplotlib.style.use('ggplot')
from sklearn.preprocessing import LabelEncoder

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [14]:
path = "/content/drive/MyDrive/heart_data/"
file = "heart_raw.csv"

df = pd.read_csv(os.path.join(path, file))

### data wrangling & EDA

In [15]:
df.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


In [18]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Age,918.0,53.510893,9.432617,28.0,47.00,54.0,60.0,77.0
RestingBP,918.0,132.396514,18.514154,0.0,120.00,130.0,140.0,200.0
Cholesterol,918.0,198.799564,109.384145,0.0,173.25,223.0,267.0,603.0
FastingBS,918.0,0.233115,0.423046,0.0,0.00,0.0,0.0,1.0
MaxHR,918.0,136.809368,25.460334,60.0,120.00,138.0,156.0,202.0
Oldpeak,918.0,0.887364,1.066570,-2.6,0.00,0.6,1.5,6.2
HeartDisease,918.0,0.553377,0.497414,0.0,0.00,1.0,1.0,1.0


In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 918 entries, 0 to 917
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Age             918 non-null    int64  
 1   Sex             918 non-null    object 
 2   ChestPainType   918 non-null    object 
 3   RestingBP       918 non-null    int64  
 4   Cholesterol     918 non-null    int64  
 5   FastingBS       918 non-null    int64  
 6   RestingECG      918 non-null    object 
 7   MaxHR           918 non-null    int64  
 8   ExerciseAngina  918 non-null    object 
 9   Oldpeak         918 non-null    float64
 10  ST_Slope        918 non-null    object 
 11  HeartDisease    918 non-null    int64  
dtypes: float64(1), int64(6), object(5)
memory usage: 86.2+ KB


### Column Type -- Category/Num

In [22]:
#change the object columns into strings
str_cols = df.select_dtypes(include='object').columns
print(str_cols)
df[str_cols] = df[str_cols].astype('string')

Index(['Sex', 'ChestPainType', 'RestingECG', 'ExerciseAngina', 'ST_Slope'], dtype='object')


In [38]:
#filter out num_cols
all_cols = df.columns
target = ['HeartDisease']
num_cols = set(all_cols) - set(str_cols) - set(target)

print(num_cols)

{'MaxHR', 'Age', 'Oldpeak', 'Cholesterol', 'RestingBP', 'FastingBS'}


### EDA


### Correlation Matrix

In [34]:
import plotly.express as px
px.imshow(df.corr(numeric_only = True),title="Correlation Plot of the Heat Failure Prediction")

#need to include numeric_only set as true, otherwise error
#Lighter shades represents positive correlation
#Darker shades represents negative correlation

### Helper Functions Data Clearning

In [44]:
#seperate columns by type

def col_type(df, target):
    str_cols = df.select_dtypes(include='object').columns
    df[str_cols] = df[str_cols].astype('string')

    target = target
    num_cols = set(all_cols) - set(str_cols) - set(target)

    print(f"string columns are {str_cols} \n")
    print(f"target columns are {target} \n")
    print(f"numerical columns are {num_cols}")

    return str_cols, num_cols, target